```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Dependencies](#dependencies)**
2. **[Input Layer](#input-layer)**
3. **[Intermediate Layer](#intermediate-layer)**
4. **[Ouput Layer](#output-layer)**
5. **[ARCHIVE](#archive)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Functions.ipynb",
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb",
        "src/DataFactory/Utils/CohortAgeGroupMapping.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL]'))
CREATE TABLE [VWSSTAGE].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[VERSION] VARCHAR(50) NULL,
	[DATE_OF_REPORT] VARCHAR(50) NULL,
	[DATE_OF_STATISTICS] VARCHAR(50) NULL,
	[BIRTH_YEAR] VARCHAR(50) NULL,
	[AGEGROUP] VARCHAR(50) NULL,
	[VACCINATION_CAMPAIGN] VARCHAR(255) NULL,
	[PERCENTAGE_COVERAGE] VARCHAR(255) NULL
);
ELSE
BEGIN
    IF NOT EXISTS (SELECT * FROM SYS.COLUMNS 
		WHERE [NAME] IN (N'VERSION')
		AND [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL]'))
			BEGIN
				ALTER TABLE [VWSSTAGE].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL]
				ADD 
					[VERSION] VARCHAR(50) NULL;
			END
	
	IF NOT EXISTS (SELECT * FROM SYS.COLUMNS 
		WHERE [NAME] IN (N'AGEGROUP')
		AND [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL]'))
			BEGIN
				ALTER TABLE [VWSSTAGE].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL]
				ADD 
					[AGEGROUP] VARCHAR(50) NULL;
			END
	
	IF NOT EXISTS
	(
		SELECT *
		FROM INFORMATION_SCHEMA.COLUMNS
		WHERE table_name = 'RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL'
			AND column_name = 'PERCENTAGE_COVERAGE'
			AND TABLE_SCHEMA = 'VWSSTAGE'
	)
	BEGIN
		EXEC sp_rename '[VWSSTAGE].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL].[PERCENTAGE]', '[PERCENTAGE_COVERAGE]', 'COLUMN'
	END;
END
GO

# **INTERMEDIATE LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSINTER].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL]'))
CREATE TABLE [VWSINTER].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_OF_REPORT] DATETIME,
	[DATE_OF_STATISTICS] DATETIME,
	[BIRTH_YEAR] VARCHAR(50),
	[AGE_GROUP] VARCHAR(50) NULL,
	[VACCINATION_SERIES] VARCHAR(255),
	[PERCENTAGE_COVERAGE] FLOAT
);
ELSE
BEGIN
	IF NOT EXISTS (SELECT * FROM SYS.COLUMNS 
		WHERE [NAME] IN (N'AGEGROUP')
		AND [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL]'))
			BEGIN
				ALTER TABLE [VWSINTER].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL]
				ADD 
					[AGE_GROUP] VARCHAR(50) NULL;
			END

	IF NOT EXISTS
	(
		SELECT *
		FROM INFORMATION_SCHEMA.COLUMNS
		WHERE table_name = 'RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL'
			AND column_name = 'VACCINATION_SERIES'
			AND TABLE_SCHEMA = 'VWSINTER'
	)
	BEGIN
		EXEC sp_rename '[VWSINTER].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL].[VACCINATION_CAMPAIGN]', '[VACCINATION_SERIES]', 'COLUMN'
	END;

	IF NOT EXISTS
	(
		SELECT *
		FROM INFORMATION_SCHEMA.COLUMNS
		WHERE table_name = 'RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL'
			AND column_name = 'PERCENTAGE_COVERAGE'
			AND TABLE_SCHEMA = 'VWSINTER'
	)
	BEGIN
		EXEC sp_rename '[VWSINTER].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL].[PERCENTAGE]', '[PERCENTAGE_COVERAGE]', 'COLUMN'
	END;
END
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO INTERMEDIATE.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_IL_RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL]
AS
BEGIN
    INSERT INTO [VWSINTER].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL] (
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [BIRTH_YEAR],
        [AGE_GROUP],
        [VACCINATION_CAMPAIGN],
        [PERCENTAGE_COVERAGE]
    )
    SELECT
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_OF_REPORT]),
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_OF_STATISTICS]),
        TRIM([BIRTH_YEAR]),
        TRIM([AGEGROUP]),
        UPPER(TRIM([VACCINATION_CAMPAIGN])),
        REPLACE([PERCENTAGE_COVERAGE], ',', '.')
    FROM
        [VWSSTAGE].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL])
END;
GO

# **OUTPUT LAYER**

---

## **<span style='color:teal'>VACCINATION SHOT COVERAGES NL</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSDEST].[RIVM_VACCINATION_SHOT_COVERAGES_NL_AUTUMN_2022]'))
CREATE TABLE [VWSDEST].[RIVM_VACCINATION_SHOT_COVERAGES_NL_AUTUMN_2022] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [DATE_OF_STATISTICS] DATETIME,
    [DATE_OF_REPORT] DATETIME,
	[AGE_GROUP] VARCHAR(50) NULL,
    [BIRTH_YEAR] VARCHAR(50),
    [AUTUMN_2022_VACCINATED] FLOAT NULL,
    [PERCENTAGE_COMPLETED] FLOAT NULL, -- not used anymore
    [PERCENTAGE_PARTLY] FLOAT NULL -- not used anymore
);
GO

-- 2) CREATE INDEX(S).....
DROP INDEX IF EXISTS [NCIX_DLI_RIVM_VACCINATION_SHOT_COVERAGES_NL_AUTUMN_2022] 
	ON [VWSDEST].[RIVM_VACCINATION_SHOT_COVERAGES_NL_AUTUMN_2022]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_RIVM_VACCINATION_SHOT_COVERAGES_NL_AUTUMN_2022]
    ON [VWSDEST].[RIVM_VACCINATION_SHOT_COVERAGES_NL_AUTUMN_2022] (
		[DATE_LAST_INSERTED]
	)
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): INTERMEDIATE TO DESTINATION.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_OL_RIVM_VACCINATION_SHOT_COVERAGES_PER_CAMPAIGN_NL]
AS
BEGIN
    WITH CTE AS (
        SELECT
            [DATE_OF_STATISTICS],
            [DATE_OF_REPORT],
            [AGE_GROUP],
            [BIRTH_YEAR],
            [VACCINATION_SERIES],
            [PERCENTAGE_COVERAGE]
        FROM
            [VWSINTER].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL])
    )
    INSERT INTO [VWSDEST].[RIVM_VACCINATION_SHOT_COVERAGES_NL_AUTUMN_2022] (
        [DATE_OF_STATISTICS],
        [DATE_OF_REPORT],
        [AGE_GROUP],
        [BIRTH_YEAR],
        [AUTUMN_2022_VACCINATED],
        [PERCENTAGE_PARTLY],
        [PERCENTAGE_COMPLETED]
    )
    SELECT
        [DATE_OF_STATISTICS],
        [DATE_OF_REPORT],
        [AGE_GROUP],
        [BIRTH_YEAR],
        [202209_CAMPAIGN],
        NULL AS [PERCENTAGE_PARTLY], -- not used anymore
        NULL AS [PERCENTAGE_COMPLETED] -- not used anymore
    FROM CTE
    PIVOT (
        MAX([PERCENTAGE_COVERAGE])
        FOR [VACCINATION_CAMPAIGN] 
        IN (
            [202209_CAMPAIGN]
        )
    ) PivotTable
END;
GO

### **<span style='color:cadetblue'>VIEWS</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VACCINATION_SHOT_COVERAGES_PER_CAMPAIGN_NL] AS
    WITH CTE AS (
        SELECT
            [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX],
            [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS]) AS [DATE_UNIX],
            [DATE_OF_REPORT],
            [AGE_GROUP],
            REPLACE(REPLACE([BIRTH_YEAR],'<=', '-'),'<', '-') AS [BIRTH_YEAR],
            CAST([PERCENTAGE_PARTLY] AS [NUMERIC](10,1)) AS [VACCINATION_COVERAGE_PARTLY],
            CAST([PERCENTAGE_COMPLETED] AS [NUMERIC](10,1)) AS [VACCINATION_COVERAGE_COMPLETED]
        FROM [VWSDEST].[RIVM_VACCINATION_SHOT_COVERAGES_PER_CAMPAIGN_NL] WITH(NOLOCK)
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_VACCINATION_SHOT_COVERAGES_PER_CAMPAIGN_NL] WITH(NOLOCK))
    ),
    COVERAGE_12_PLUS AS (
        SELECT
            [DATE_UNIX],
            [DATE_OF_INSERTION_UNIX],
            [DATE_OF_REPORT],
            [BIRTH_YEAR],
            [VACCINATION_COVERAGE_PARTLY],
            [VACCINATION_COVERAGE_COMPLETED]
        FROM CTE
        WHERE [DATE_UNIX] = (SELECT MAX([DATE_UNIX]) FROM CTE)
            AND [AGE_GROUP] = '12+'
    ),
    COVERAGE_18_PLUS AS (
        SELECT
            [DATE_UNIX],
            [DATE_OF_INSERTION_UNIX],
            [DATE_OF_REPORT],
            [BIRTH_YEAR],
            [VACCINATION_COVERAGE_PARTLY],
            [VACCINATION_COVERAGE_COMPLETED]
        FROM CTE
        WHERE [DATE_UNIX] = (SELECT MAX([DATE_UNIX]) FROM CTE)
            AND [AGE_GROUP] = '18+'
    )
    SELECT
        T1.[BIRTH_YEAR] AS [AGE_12_PLUS_BIRTHYEAR],
        T1.[VACCINATION_COVERAGE_PARTLY] AS [AGE_12_PLUS_HAS_ONE_SHOT],
        T1.[VACCINATION_COVERAGE_COMPLETED] AS [AGE_12_PLUS_FULLY_VACCINATED],
        T2.[BIRTH_YEAR] AS [AGE_18_PLUS_BIRTHYEAR],
        T2.[VACCINATION_COVERAGE_PARTLY] AS [AGE_18_PLUS_HAS_ONE_SHOT],
        T2.[VACCINATION_COVERAGE_COMPLETED] AS [AGE_18_PLUS_FULLY_VACCINATED],
        T2.[DATE_UNIX],
        T2.[DATE_OF_INSERTION_UNIX]
    FROM COVERAGE_12_PLUS T1
    INNER JOIN COVERAGE_18_PLUS T2 ON T1.[DATE_OF_REPORT] = T2.[DATE_OF_REPORT]
GO

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VACCINATION_SHOT_COVERAGES_NL] AS
    WITH CTE AS (
        SELECT
            [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX],
            [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_REPORT]) AS [DATE_UNIX],
            [DATE_OF_REPORT],
            [AGE_GROUP],
            REPLACE(REPLACE([BIRTH_YEAR],'<=', '-'),'<', '-') AS [BIRTH_YEAR],
            CAST([AUTUMN_2022_VACCINATED] AS [NUMERIC](10,1)) AS [VACCINATION_COVERAGE_AUTUMN_2022],
            CAST([PERCENTAGE_COMPLETED] AS [NUMERIC](10,1)) AS [VACCINATION_COVERAGE_BASISSERIE],
            CAST([PERCENTAGE_PARTLY] AS [NUMERIC](10,1)) AS [VACCINATION_COVERAGE_PARTLY]
        FROM [VWSDEST].[RIVM_VACCINATION_SHOT_COVERAGES_NL_AUTUMN_2022] WITH(NOLOCK)
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_VACCINATION_SHOT_COVERAGES_NL_AUTUMN_2022] WITH(NOLOCK))
    ),
    COVERAGE_12_PLUS AS (
        SELECT
            [DATE_UNIX],
            [DATE_OF_INSERTION_UNIX],
            [DATE_OF_REPORT],
            [BIRTH_YEAR],
            MAX([VACCINATION_COVERAGE_AUTUMN_2022]) as VACCINATION_COVERAGE_AUTUMN_2022,
            MAX([VACCINATION_COVERAGE_BASISSERIE]) as VACCINATION_COVERAGE_BASISSERIE,
            MAX([VACCINATION_COVERAGE_PARTLY]) as VACCINATION_COVERAGE_PARTLY
        FROM CTE
        WHERE [DATE_UNIX] = (SELECT MAX([DATE_UNIX]) FROM CTE)
            AND [AGE_GROUP] = '12+'
        GROUP BY [DATE_UNIX],
                 [DATE_OF_INSERTION_UNIX],
                 [DATE_OF_REPORT],
                 [BIRTH_YEAR]
    ),
    COVERAGE_18_PLUS AS (
        SELECT
            [DATE_UNIX],
            [DATE_OF_INSERTION_UNIX],
            [DATE_OF_REPORT],
            [BIRTH_YEAR],
            MAX([VACCINATION_COVERAGE_AUTUMN_2022]) as VACCINATION_COVERAGE_AUTUMN_2022,
            MAX([VACCINATION_COVERAGE_BASISSERIE]) as VACCINATION_COVERAGE_BASISSERIE,
            MAX([VACCINATION_COVERAGE_PARTLY]) as VACCINATION_COVERAGE_PARTLY
        FROM CTE
        WHERE [DATE_UNIX] = (SELECT MAX([DATE_UNIX]) FROM CTE)
            AND [AGE_GROUP] = '18+'
        GROUP BY [DATE_UNIX],
                 [DATE_OF_INSERTION_UNIX],
                 [DATE_OF_REPORT],
                 [BIRTH_YEAR]            
    ),
    COVERAGE_60_PLUS AS (
        SELECT
            [DATE_UNIX],
            [DATE_OF_INSERTION_UNIX],
            [DATE_OF_REPORT],
            [BIRTH_YEAR],
            MAX([VACCINATION_COVERAGE_AUTUMN_2022]) as VACCINATION_COVERAGE_AUTUMN_2022,
            MAX([VACCINATION_COVERAGE_BASISSERIE]) as VACCINATION_COVERAGE_BASISSERIE,
            MAX([VACCINATION_COVERAGE_PARTLY]) as VACCINATION_COVERAGE_PARTLY
        FROM CTE
        WHERE [DATE_UNIX] = (SELECT MAX([DATE_UNIX]) FROM CTE)
            AND [AGE_GROUP] = '60+'
        GROUP BY [DATE_UNIX],
                 [DATE_OF_INSERTION_UNIX],
                 [DATE_OF_REPORT],
                 [BIRTH_YEAR]
    )
    SELECT
        T1.[BIRTH_YEAR] AS [AGE_12_PLUS_BIRTHYEAR],
        T1.[VACCINATION_COVERAGE_AUTUMN_2022] AS [AGE_12_PLUS_AUTUMN_2022_VACCINATED],
        T1.[VACCINATION_COVERAGE_BASISSERIE] AS [AGE_12_PLUS_FULLY_VACCINATED],
        T1.[VACCINATION_COVERAGE_PARTLY] AS [AGE_12_PLUS_HAS_ONE_SHOT],
        T2.[BIRTH_YEAR] AS [AGE_18_PLUS_BIRTHYEAR],
        T2.[VACCINATION_COVERAGE_BASISSERIE] AS [AGE_18_PLUS_FULLY_VACCINATED],
        T2.[VACCINATION_COVERAGE_PARTLY] AS [AGE_18_PLUS_HAS_ONE_SHOT],
        T3.[VACCINATION_COVERAGE_AUTUMN_2022] AS [AGE_60_PLUS_AUTUMN_2022_VACCINATED],
        T3.[BIRTH_YEAR] AS [AGE_60_PLUS_BIRTHYEAR],
        T2.[DATE_UNIX],
        T2.[DATE_OF_INSERTION_UNIX]
    FROM COVERAGE_12_PLUS T1
    LEFT JOIN COVERAGE_18_PLUS T2 ON T1.[DATE_OF_REPORT] = T2.[DATE_OF_REPORT]
    LEFT JOIN COVERAGE_60_PLUS T3 ON T1.[DATE_OF_REPORT] = T3.[DATE_OF_REPORT]
GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION</span>**

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
/*
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VACCINATION_SHOT_COVERAGES_PER_CAMPAIGN_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_coverage_per_age_group_estimated',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 0
    WHEN 'acceptance' THEN 0
    ELSE 0
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO
*/

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VACCINATION_SHOT_COVERAGES_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_coverage_per_age_group_estimated',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

## **<span style='color:teal'>BOOSTER SHOT COVERAGES NL</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSDEST].[RIVM_BOOSTER_SHOT_COVERAGES_NL]'))
CREATE TABLE [VWSDEST].[RIVM_BOOSTER_SHOT_COVERAGES_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [DATE_OF_STATISTICS] DATETIME,
    [DATE_OF_REPORT] DATETIME,
	[AGE_GROUP] VARCHAR(50) NULL,
    [BIRTH_YEAR] VARCHAR(50),
    [PERCENTAGE] FLOAT
);
GO

-- 2) CREATE INDEX(S).....
DROP INDEX IF EXISTS [NCIX_DLI_RIVM_BOOSTER_SHOT_COVERAGES_NL] 
	ON [VWSDEST].[RIVM_BOOSTER_SHOT_COVERAGES_NL]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_RIVM_BOOSTER_SHOT_COVERAGES_NL]
    ON [VWSDEST].[RIVM_BOOSTER_SHOT_COVERAGES_NL] (
		[DATE_LAST_INSERTED]
	)
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): INTERMEDIATE TO DESTINATION.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_OL_RIVM_BOOSTER_SHOT_COVERAGES_NL]
AS
BEGIN
    INSERT INTO [VWSDEST].[RIVM_BOOSTER_SHOT_COVERAGES_NL] (
        [DATE_OF_STATISTICS],
        [DATE_OF_REPORT],
        [AGE_GROUP],
        [BIRTH_YEAR],
        [PERCENTAGE]
    )
    SELECT
        T1.[DATE_OF_STATISTICS],
        T1.[DATE_OF_REPORT],
        T2.[AGE_GROUP],
        T1.[BIRTH_YEAR],
        T1.[PERCENTAGE]
    FROM
        [VWSINTER].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL] T1
    LEFT JOIN
        [VWSSTATIC].[COHORT_TO_AGE_GROUPS_MAPPING] T2 WITH(NOLOCK) ON T1.[BIRTH_YEAR] = T2.[COHORT]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[RIVM_VACCINATION_COVERAGES_PER_CAMPAIGN_NL])
        AND [VACCINATION_CAMPAIGN] IN ('BOOSTER_COVERAGE')
END;
GO

### **<span style='color:cadetblue'>VIEWS</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_BOOSTER_SHOT_COVERAGES_NL] AS
    WITH CTE AS (
        SELECT
            [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX],
            [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS]) AS [DATE_UNIX],            
            -- UPPER(DATENAME(dw, [DATE_OF_STATISTICS])) AS [DAY_OF_WEEK],
            [AGE_GROUP],
            CAST([PERCENTAGE] AS [NUMERIC](10,1)) AS [PERCENTAGE]
        FROM [VWSDEST].[RIVM_BOOSTER_SHOT_COVERAGES_NL] WITH(NOLOCK)
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_BOOSTER_SHOT_COVERAGES_NL] WITH(NOLOCK))
            AND [AGE_GROUP] IN ('18+', '12+')
    )
    SELECT
        [AGE_GROUP],
        [PERCENTAGE],
        [DATE_OF_INSERTION_UNIX],
        [DATE_UNIX]
    FROM CTE
    WHERE [DATE_UNIX] = (
        SELECT 
            MAX([DATE_UNIX]) 
        FROM CTE 
        -- WHERE [DAY_OF_WEEK] = 'SUNDAY'
    )
GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION</span>**

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_BOOSTER_SHOT_COVERAGES_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'booster_coverage',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 2,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO